In [1]:
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# load PR Newswire Annotated Dataset
pr_newswire = pd.read_csv("../data/pr-newswire.csv")

In [3]:
raw_news_stories = pr_newswire["data"]

# Establish data and target for vectorization
stories = raw_news_stories
classifications = list(pr_newswire["target"])

In [4]:
predictions = []
for story in list(stories):
    classification = []
    if "cash dividend" in story:
        classification.append("cash-dividend")
    if "stock split" in story:
        classification.append("stock-split")
    if "merger" in story or "acquisition" in story:
        classification.append("merger-acquisition")   
    if "equity offering" in story:
        classification.append("equity-offering")
    if "bankruptcy" in story:
        classification.append("bankruptcy")   
    
    if len(classification) == 1:
        prediction = classification[0]
    else:
        prediction = "ambiguous"
        
    predictions.append(prediction)


In [5]:
# Calculate Correct Classifications
correct_values = sum(
    1 for cls, pred in zip(classifications, predictions) if cls == pred
)

# Total Values
total_values = len(classifications)

# Accuracy
print(f"Accuracy Score: {correct_values / total_values}")

Accuracy Score: 0.5233333333333333
